In [1]:
import pandas as pd
import numpy as np
import requests
# from fuzzywuzzy import fuzz
import json
import os
import re
import geopandas as gp
import util

In [2]:
df = pd.read_csv("exact_matches_1_final.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'exact_matches_1_final.csv'

In [8]:
len(df)

5436

In [59]:
df=df.drop_duplicates(subset='UBINumber').reset_index()

In [60]:
df.tail()

,index,SearchTerm,BusinessName,PotentialRelatedCompany,UBINumber,Address,Status,Principals,isMatch,notes
327,5431,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,423 THIRD AVENUE WEST LLC,1315540,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",Active,"['JOHN M. GREELEY', 'MARTIN SELIG']",NaN,NaN
328,5432,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,333 LEASING LLC,1448725,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",Active,"['JOHN M GREELEY', 'MARTIN SELIG']",NaN,NaN
329,5433,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,1399595,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",Active,"['JOHN M. GREELEY', 'MARTIN SELIG']",NaN,NaN
330,5434,300 FIFTH AVENUE LLC,300 FIFTH AVENUE LLC,1015 SECOND AVENUE LLC,1470,"1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...",Administratively Dissolved,['MARTIN SELIG'],NaN,NaN
331,5435,KAR STANDARD LLC,KAR STANDARD LLC,KAR STANDARD LLC,763141,"1 FEDERAL ST FL 17, BOSTON, MA, 02110-2003, UN...",Active,['KAONOULU RANCH LLLP'],NaN,NaN


In [67]:
groups = df.groupby("BusinessName")
for name, group in groups: 
    print("\n", name, len(group))
    matched_result = group[group["BusinessName"]==group["PotentialRelatedCompany"]]
    matched_principals = matched_result["Principals"].values[0].strip('][').split(', ')
    # print(matched_principals)
    for idx, row in group.iterrows(): 
        potential_principals = row["Principals"].strip('][').split(', ')
        intersect = set(matched_principals) & set(potential_principals)
        # print(idx)
        df.at[idx,"Overlap"] = len(intersect)/len(matched_principals)
        # print(df.iloc[idx])



 300 FIFTH AVENUE LLC 77
254
index                                                                   5358
SearchTerm                                              300 FIFTH AVENUE LLC
BusinessName                                            300 FIFTH AVENUE LLC
PotentialRelatedCompany                    YESLER INVESTMENT COMPANY, L.L.C.
UBINumber                                                            1082285
Address                    1000 2ND AVE STE 1800, SEATTLE, WA, 98104-3619...
Status                                            Administratively Dissolved
Principals                                                  ['MARTIN SELIG']
isMatch                                                                  NaN
notes                                                                    NaN
Overlap                                                                  0.5
Name: 254, dtype: object
255
index                                                                   5359
SearchTerm       

In [69]:
df.columns

Index(['index', 'SearchTerm', 'BusinessName', 'PotentialRelatedCompany',
       'UBINumber', 'Address', 'Status', 'Principals', 'isMatch', 'notes',
       'Overlap'],
      dtype='object')

In [70]:
df = df[['index', 'SearchTerm', 'BusinessName', 'PotentialRelatedCompany',
       'UBINumber', 'Address', 'Status', 'Principals','Overlap', 'isMatch', 'notes']]

In [71]:
df.head()

,index,SearchTerm,BusinessName,PotentialRelatedCompany,UBINumber,Address,Status,Principals,Overlap,isMatch,notes
0,0,BPP 800 FIFTH PROPERTY OWNER LLC,BPP 800 FIFTH PROPERTY OWNER LLC,"WESTERN C REIT WA, LLC",519171,"233 S WACKER DR, SUITE 4700, CHICAGO, IL, 6060...",Active,"['A.J. AGARWAL', 'ADAM LESLIE', 'ANDREA DRASIT...",0.734940,NaN,NaN
1,1,BPP 800 FIFTH PROPERTY OWNER LLC,BPP 800 FIFTH PROPERTY OWNER LLC,"WESTERN B NORTHWEST WA, LLC",519167,"233 S. WACKER DRIVE, SUITE 4700, CHICAGO, IL, ...",Active,"['A.J. AGARWAL', 'ADAM LESLIE', 'ANDREA DRASIT...",0.734940,NaN,NaN
2,2,BPP 800 FIFTH PROPERTY OWNER LLC,BPP 800 FIFTH PROPERTY OWNER LLC,"SECURITY MANAGEMENT DIVISION, LLC",1583621,"3225 MCLEOD DR STE 100, LAS VEGAS, NV, 89121-2...",Active,"['RICHARD REYES', 'WENDY BALELO']",0.012048,NaN,NaN
3,3,BPP 800 FIFTH PROPERTY OWNER LLC,BPP 800 FIFTH PROPERTY OWNER LLC,PP TANGO WA LLC,1302391,"233 S WACKER DR STE 4700, CHICAGO, IL, 60606-6...",Active,"['A.J. AGARWAL', 'ADAM LESLIE', 'ANDREA DRASIT...",0.759036,NaN,NaN
4,4,BPP 800 FIFTH PROPERTY OWNER LLC,BPP 800 FIFTH PROPERTY OWNER LLC,PIH OLYMPIA LLC,1141183,"233 S WACKER DR STE 4700, CHICAGO, IL, 60606-6...",Active,"['ADAM LESLIE', 'AJ AGARWAL', 'ANDREA DRASITES...",0.698795,NaN,NaN


In [72]:
df.to_csv("exact_matches_1_final_unduped.csv")